In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


#**LOADING DATA**

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:


words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

340 documents
93 classes ['addiction_recovery', 'anger_management', 'anger_management_strategies', 'animal_assisted_therapy', 'anxiety_management', 'art_therapy', 'bipolar_disorder', 'bipolar_disorder_management', 'body_image', 'borderline_personality_disorder', 'chatbot_info', 'children_mental_health', 'cognitive_distortions', 'counseling_services', 'cultural_sensitivity', 'depression_symptoms', 'eating_disorder_recovery', 'eating_disorders', 'emergency', 'emergency_resources', 'emotional_regulation', 'exercise_for_mental_health', 'family_support', 'gender_identity', 'goodbye', 'greeting', 'grief_and_loss', 'information', 'insomnia_management', 'insurance_coverage', 'loneliness_and_depression', 'loneliness_effects', 'medication', 'medication_info', 'medication_management', 'meditation', 'meditation_practices', 'mental_health', 'mental_health_apps', 'mental_health_books', 'mental_health_law', 'mental_health_stigma', 'mental_health_workplace', 'mental_health_workshops', 'mindfulness_med

# **TRAINING DATA**

In [4]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


<ipython-input-4-dd41517aad3b>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)




/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
68/68 [==============================] - 1s 1ms/step - loss: 4.5546 - accuracy: 0.0088
Epoch 2/200
68/68 [==============================] - 0s 1ms/step - loss: 4.5027 - accuracy: 0.0324
Epoch 3/200
68/68 [==============================] - 0s 1ms/step - loss: 4.4685 - accuracy: 0.0529
Epoch 4/200
68/68 [==============================] - 0s 1ms/step - loss: 4.4131 - accuracy: 0.0559
Epoch 5/200
68/68 [==============================] - 0s 1ms/step - loss: 4.3654 - accuracy: 0.0735
Epoch 6/200
68/68 [==============================] - 0s 1ms/step - loss: 4.2612 - accuracy: 0.0676
Epoch 7/200
68/68 [==============================] - 0s 1ms/step - loss: 4.1371 - accuracy: 0.1088
Epoch 8/200
68/68 [==============================] - 0s 1ms/step - loss: 4.0544 - accuracy: 0.0824
Epoch 9/200
68/68 [==============================] - 0s 1ms/step - loss: 3.8455 - accuracy: 0.1353
Epoch 10/200
68/68 [==============================] - 0s 1ms/step - loss: 3.7607 - accuracy: 0.1500
Epoch 11/

#**PREDICTION**

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [8]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [9]:
chatbot_response("how can i deal with anxiety")

1/1 [==============================] - 0s 67ms/step


'Anxiety management techniques include deep breathing, progressive muscle relaxation, and positive self-talk. We can guide you through these practices to reduce anxiety effectively.'